# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,severo-kurilsk,50.6789,156.1250,-3.22,78,98,3.33,RU,1675639538
1,1,cidreira,-30.1811,-50.2056,23.78,76,64,4.39,BR,1675639538
2,2,bongandanga,1.5000,21.0500,24.12,33,52,0.76,CD,1675639539
3,3,ushuaia,-54.8000,-68.3000,7.79,71,56,3.67,AR,1675638729
4,4,dikson,73.5069,80.5464,-19.10,97,100,0.74,RU,1675639539


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = 'City',
    alpha = 0.75,
    tiles = "EsriNatGeo",
    size ='Humidity',
    
    
    
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_criteria = city_data_df[(city_data_df['Max Temp'] <= 18) &
                                   (city_data_df['Humidity'] > 40) &
                                   (city_data_df['Cloudiness'] <= 30) &
                                   (city_data_df['Wind Speed'] <= 15)]
# Drop any rows with null values
ideal_criteria_df = ideal_criteria.dropna()

# Display sample data
ideal_criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,guerrero negro,27.9769,-114.0611,17.47,67,9,8.37,MX,1675638996
22,22,hasaki,35.7333,140.8333,7.34,77,1,3.64,JP,1675639546
25,25,qaanaaq,77.4840,-69.3632,-18.35,55,5,2.97,GL,1675639547
36,36,punta arenas,-53.1500,-70.9167,11.06,62,0,7.20,CL,1675639552
47,47,xining,36.6167,101.7667,-5.35,61,1,0.83,CN,1675639556
...,...,...,...,...,...,...,...,...,...,...
542,542,manama,26.2154,50.5832,17.74,77,0,1.54,BH,1675639783
547,547,nantucket,41.2835,-70.0995,7.08,87,0,6.17,US,1675639785
557,557,lalganj,25.8667,85.1833,13.97,73,0,2.78,IN,1675639648
575,575,rajapur,25.3833,81.1500,13.81,52,0,0.94,IN,1675639396


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_criteria_df[['City', 'Country', 'Lat','Lng', "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,guerrero negro,MX,27.9769,-114.0611,67,
22,hasaki,JP,35.7333,140.8333,77,
25,qaanaaq,GL,77.4840,-69.3632,55,
36,punta arenas,CL,-53.1500,-70.9167,62,
47,xining,CN,36.6167,101.7667,61,
...,...,...,...,...,...,...
542,manama,BH,26.2154,50.5832,77,
547,nantucket,US,41.2835,-70.0995,87,
557,lalganj,IN,25.8667,85.1833,73,
575,rajapur,IN,25.3833,81.1500,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
categories = ('internet_access.free','accommodation.hotel')
params = {
    'radius': radius, 'categories': categories , 'apiKey': geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"  # YOUR CODE HERE
    params["bias"] = f"proximity:{Lng},{Lat}" # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
guerrero negro - nearest hotel: Plaza sal paraiso
hasaki - nearest hotel: 大新旅館
qaanaaq - nearest hotel: Hotel Qaanaaq
punta arenas - nearest hotel: Hotel Lacolet
xining - nearest hotel: Qinghai hotel
hamilton - nearest hotel: North Vista Manor
erenhot - nearest hotel: Oulen hotel
hami - nearest hotel: Хами
dalmau - nearest hotel: No hotel found
aguimes - nearest hotel: Hotel Villa de Agüimes
tura - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
fairbanks - nearest hotel: Westmark Fairbanks Hotel
mach - nearest hotel: No hotel found
marrakesh - nearest hotel: No hotel found
ancud - nearest hotel: Cabañas
clarksdale - nearest hotel: No hotel found
toccoa - nearest hotel: No hotel found
inuvik - nearest hotel: Capital Suites
hovd - nearest hotel: Khovd hotel
mayo - nearest hotel: Pirates Cove Restaurant and Dock Bar
glendive - nearest hotel: La Quinta Inn & Suites by Wyndham Glendive
fukue - nearest hotel: 五島第一ホテル
cedar falls - nearest hotel: Hamp

,City,Country,Lat,Lng,Humidity,Hotel Name
11,guerrero negro,MX,27.9769,-114.0611,67,Plaza sal paraiso
22,hasaki,JP,35.7333,140.8333,77,大新旅館
25,qaanaaq,GL,77.4840,-69.3632,55,Hotel Qaanaaq
36,punta arenas,CL,-53.1500,-70.9167,62,Hotel Lacolet
47,xining,CN,36.6167,101.7667,61,Qinghai hotel
...,...,...,...,...,...,...
542,manama,BH,26.2154,50.5832,77,Hotel Mansouri Mansions
547,nantucket,US,41.2835,-70.0995,87,Periwinkle Bed and Breakfast
557,lalganj,IN,25.8667,85.1833,73,No hotel found
575,rajapur,IN,25.3833,81.1500,52,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",    
    size = "Humidity",
    color = "City",
    
    hover_cols=["Country", "Hotel Name"]
)


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)